In [3]:
import rasterio
import matplotlib
import numpy as np
from sedona.raster_utils import SedonaUtils

In [4]:
from sedona.spark import *

additional_packages = [
    'org.apache.hadoop:hadoop-aws:3.3.4',
    'org.apache.hadoop:hadoop-client-api:3.3.4',
    'org.apache.hadoop:hadoop-common:3.3.4',
    'org.apache.sedona:sedona-spark-3.5_2.12:1.7.0',
    'org.datasyslab:geotools-wrapper:1.7.0-28.5',
    'uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.4'
]

config = (
    SedonaContext.builder()
    .config("spark.jars.packages", ",".join(additional_packages))\
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", "sedona") \
    .config("spark.hadoop.fs.s3a.secret.key", "sedona_password") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config(
      "spark.hadoop.fs.s3a.aws.credentials.provider",
      "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.driver.memory", "2G")
    .getOrCreate()
)

sedona = SedonaContext.create(config)

sc = sedona.sparkContext

24/12/18 19:39:05 WARN Utils: Your hostname, Pawels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.115 instead (on interface en0)
24/12/18 19:39:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/pawelkocinski/Desktop/projects/apache-sedona-book/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/pawelkocinski/.ivy2/cache
The jars for the packages stored in: /Users/pawelkocinski/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client-api added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
uk.co.gresearch.spark#spark-extension_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-84dcb853-d67f-4297-aa6e-7755ccdee824;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in local-m2-cache
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-s

In [5]:
geotiff_df = sedona\
    .read\
    .format("binaryFile")\
    .load("s3a://sedona/raster/japan_landsat_part.tif")

24/12/18 19:39:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
geotiff_df\
    .selectExpr("RS_FromGeoTiff(content) AS raster")\
    .createOrReplaceTempView("japan")

In [7]:
sedona\
    .sql("SELECT EXPLODE(RS_Tile(raster, 10, 10)) AS tile FROM japan")\
    .createOrReplaceTempView("tiles")

In [8]:
wkt = 'POLYGON ((747726.6770902255084366 3856119.2088216841220856, 741397.5797568620182574 3853730.8702053208835423, 734590.8147002257173881 3856238.6257525025866926, 729455.8866750439628959 3861373.5537776844576001, 732321.8930146803613752 3866030.8140795934014022, 735784.9840084075694904 3868657.9865575931034982, 733754.8961844984441996 3871046.3251739568077028, 729336.4697442258475348 3869374.4881425024941564, 723365.6232033168198541 3864836.6447714115493000, 718350.1121089532971382 3861731.8045701389200985, 713692.8518070442369208 3859343.4659537752158940, 708796.7576434988295659 3862925.9738783207722008, 705572.5005114079685882 3869971.5727965934202075, 707244.3375428625149652 3876778.3378532296046615, 710468.5946749533759430 3885854.0245954110287130, 716678.2750774987507612 3892899.6235136836767197, 723007.3724108622409403 3894571.4605451384559274, 731127.7237064985092729 3896362.7145074112340808, 739725.7427254074718803 3896243.2975765927694738, 747129.5924361345823854 3894690.8774759564548731, 752264.5204613163368776 3892421.9557904112152755, 753936.3574927708832547 3889914.2002432295121253, 755011.1098701345035806 3886451.1092495019547641, 757638.2823481344385073 3885256.9399413201026618, 759310.1193795889848843 3882510.3505325019359589, 759310.1193795889848843 3879644.3441928657703102, 758354.7839330435963348 3878330.7579538659192622, 757757.6992789526702836 3876539.5039915931411088, 757877.1162097707856447 3874628.8330985023640096, 757996.5331405890174210 3872837.5791362295858562, 758593.6177946799434721 3871165.7421047748066485, 756682.9469015890499577 3870449.2405198658816516, 756563.5299707708181813 3868299.7357651386409998, 755966.4453166800085455 3864597.8109097750857472, 755369.3606625890824944 3861731.8045701389200985, 755369.3606625890824944 3859701.7167462296783924, 751309.1850147709483281 3857791.0458531389012933, 747726.6770902255084366 3856119.2088216841220856))'

In [9]:
sedona.sql(
    f"""
    WITH algebra_result AS (
        SELECT 
            RS_MapAlgebra(tile, 'D', 'out = (rast[1] - rast[0]) / (rast[1] + rast[0]);') AS raster,
            ST_SetSRID(ST_GeomFromText('{wkt}'), 32653) AS geom
        FROM tiles
    ),
    clipped AS (
        SELECT RS_Clip(raster, 1, geom, 0, false) AS raster
        FROM algebra_result
        WHERE RS_Intersects(geom, raster)
    )
    SELECT raster
    FROM clipped
    """
).createOrReplaceTempView("clipped")

In [11]:
# here is the error that need to be addresses in Sedona
sedona.sql("select * from clipped").show()

ANTLR Tool version 4.7.1 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.7.1 used for parser compilation does not match the current runtime version 4.9.3
ANTLR Tool version 4.7.1 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.7.1 used for parser compilation does not match the current runtime version 4.9.3
24/12/18 19:40:14 WARN VectorToRasterProcess: coercing double feature values to float raster values
24/12/18 19:40:14 ERROR Executor: Exception in task 0.0 in stage 3.0 (TID 2)
org.apache.spark.sql.sedona_sql.expressions.InferredExpressionException: Exception occurred while evaluating expression RS_Clip - inputs: [GridCoverage2D["genericCoverage", GeneralEnvelope[(729960.0, 3896100.0), (730260.0, 3896400.0)], DefaultProjectedCRS["WGS 84 / UTM zone 53N"]]
│   RenderedSampleDimension("genericCoverage":[-1.7976931348623157E308 ... 1.7976931348623157E308])
│     ‣ Category("genericCoverage

Py4JJavaError: An error occurred while calling o71.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 2) (192.168.0.115 executor driver): org.apache.spark.sql.sedona_sql.expressions.InferredExpressionException: Exception occurred while evaluating expression RS_Clip - inputs: [GridCoverage2D["genericCoverage", GeneralEnvelope[(729960.0, 3896100.0), (730260.0, 3896400.0)], DefaultProjectedCRS["WGS 84 / UTM zone 53N"]]
│   RenderedSampleDimension("genericCoverage":[-1.7976931348623157E308 ... 1.7976931348623157E308])
│     ‣ Category("genericCoverage":[-1.7976931348623157E308 ... 1.7976931348623157E308])
└ Image=WritableRenderedImageAdapter[]
, 1, POLYGON ((747726.6770902255 3856119.208821684, 741397.579756862 3853730.870205321, 734590.8147002257 3856238.6257525026, 729455.886675044 3861373.5537776845, 732321.8930146804 3866030.8140795934, 735784.9840084076 3868657.986557